In [ ]:
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install scikit-learn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
import cuml
import cudf
from cuml.model_selection._split import train_test_split
from sklearn.model_selection import train_test_split

In [ ]:
details = pd.read_csv("garmin_edge_820/4557226804_ACTIVITY_data.csv", sep=";")

In [ ]:
# Pulizia dei dati
details = details.drop(['left_power_phase[degrees]',
                        'left_power_phase_peak[degrees]',
                        'right_power_phase[degrees]',
                        'right_power_phase_peak[degrees]',
                        'left_right_balance'], axis=1)
# convertire i valori di tempo in formato datetime
details['time'] = pd.to_datetime(details.pop('timestamp[s]'), unit='s').dt.time
details.set_index("time", inplace=True)

def convert_brackets(string):
    return string.replace('[', '(').replace(']', ')')

details.columns = [convert_brackets(col) for col in details.columns]

# calcolo il tempo trascorso dall'inizio dell'attività
details['time_since_start'] = 1
details['time_since_start'] = details['time_since_start'].cumsum().sub(1)

# calcolo le zone di frequenza cardiaca e di potenza dato il battito in input

hr_zones = [(0, 128), (129, 146), (147, 156), (157, 165),
            (166, 174), (175, 179), (180, float('inf'))]
power_zones = [(0, 157), (158, 186), (187, 200), (201, 218),
               (219, 247), (248, 287), (288, float('inf'))]

def get_zone(rate, zones):
    for zone, (lower, upper) in enumerate(zones, start=1):
        if lower <= rate <= upper:
            return zone

details['hr_zone'] = details['heart_rate(bpm)'].apply(get_zone, zones=hr_zones)
details['pwr_zone'] = details['power(watts)'].apply(get_zone, zones=power_zones)

# Calcola la differenza di altitudine tra le righe adiacenti
details['altitude_diff'] = details['altitude(m)'] - \
    details['altitude(m)'].shift(1)
details['distance_diff'] = details['distance(m)'] - \
    details['distance(m)'].shift(1)
details[['altitude_diff', 'distance_diff']] = details[[
    'altitude_diff', 'distance_diff']].fillna(0)
# Calcola la percentuale di pendenza
details['slope_percent'] = np.where(
    details['distance_diff'] == 0, 0, details['altitude_diff'] / details['distance_diff'] * 100)
details['speed(m/s)'] = details['speed(m/s)'].fillna(details['speed(m/s)'].mean())

In [ ]:
details = cudf.DataFrame(details)
X = details.drop('heart_rate(bpm)', axis=1)
y = details['heart_rate(bpm)']
X_train, X_test, y_train, y_test = cuml.model_selection.train_test_split(X, y, test_size=1/3, train_size=0.75,  random_state=42)

In [ ]:
X_cudf_train = cudf.DataFrame.from_pandas(X_train)
X_cudf_test = cudf.DataFrame.from_pandas(X_test)

y_cudf_train = cudf.Series(y_train.values)

In [ ]:
lr = cuml.LinearRegression(algorithm='svd', fit_intercept=True, normalize=True)
lr.fit(X_cudf_train,y_cudf_train)